# Notebook for Running the Image Segmentation on Colab

**Note:**
- This notebook is here for documentation reasons
- It was executed on Google Colab
- The image segmentation should be executed on a powerful GPU, e.g. with Google Colab or on a HPC
- To execute this on Colab without further adjustments, it would be necessary to recreate the folder structure and upload the custom packages from `src/models/image_segmentation`

### Preparation

**Important:** You need to be connected to a GPU, else `inplace_abn` fails to install

In [ ]:
!pip install inplace_abn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for inplace_abn: filename=inplace_abn-1.1.0-cp310-cp310-linux_x86_64.whl size=4215172 sha256=eaf3eff00db0cc86ff010e7716990d54150d7108834b7a0e126a8fb1cdfe22ff
  Stored in directory: /root/.cache/pip/wheels/c7/f2/05/97ab74dbf2ed6d592684f76d0ae8f33f66bbafa0ec5b53b416
Successfully built inplace_abn


In [ ]:
import os
from os import path

import torch
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/2024_Data_Science_Project/scripts')
from segmentation_setup.segmentation_setup import *

### Unzip images (only the first time)

In [ ]:
#os.chdir('/content/drive/My Drive/2024_Data_Science_Project/segmentation')

#import zipfile

#zipped_folder_path = 'edges_svi_sorted.zip'
#unzip_destination = 'edges'

# Unzip the folder
#with zipfile.ZipFile(zipped_folder_path, 'r') as zip_ref:
#    zip_ref.extractall(unzip_destination)

The next code block sorts images that were already sorted (e.g. in a previous notebook run) out of the source folder `edges`, so that they are not segmented again

In [ ]:
os.chdir('/content/drive/My Drive/2024_Data_Science_Project/segmentation')

import shutil

# Specify the paths to your folders
input_folder = 'edges'
output_folder = 'edges_img_output'
input_bin_folder = 'edges_bin'

# Ensure the input_bin folder exists
if not os.path.exists(input_bin_folder):
    os.makedirs(input_bin_folder)

# Iterate over the files in the output folder
for output_file in os.listdir(output_folder):
    # Construct the base filename by removing the extension
    base_name = os.path.splitext(output_file)[0]
    input_file_name = base_name + '.jpg'  # Assuming input files are in .jpg format

    # Construct the full path to the input and destination files
    input_file_path = os.path.join(input_folder, input_file_name)
    dest_file_path = os.path.join(input_bin_folder, input_file_name)

    # Check if the corresponding input file exists
    if os.path.exists(input_file_path):
        # Move the input file to the input_bin folder
        shutil.move(input_file_path, dest_file_path)
        print(f'Moved {input_file_name} to {input_bin_folder}')

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Moved edges-cluster_2737.jpg to edges_bin
Moved edges-cluster_6462.jpg to edges_bin
Moved edges-cluster_9155.jpg to edges_bin
Moved edges-cluster_2911.jpg to edges_bin
Moved edges-cluster_14861.jpg to edges_bin
Moved edges-cluster_4432.jpg to edges_bin
Moved edges-cluster_13369.jpg to edges_bin
Moved edges-cluster_17683.jpg to edges_bin
Moved edges-cluster_5272.jpg to edges_bin
Moved edges-cluster_4376.jpg to edges_bin
Moved edges-cluster_7568.jpg to edges_bin
Moved edges-cluster_10137.jpg to edges_bin
Moved edges-cluster_18016.jpg to edges_bin
Moved edges-cluster_11657.jpg to edges_bin
Moved edges-cluster_12350.jpg to edges_bin
Moved edges-cluster_6142.jpg to edges_bin
Moved edges-cluster_2760.jpg to edges_bin
Moved edges-cluster_21550.jpg to edges_bin
Moved edges-cluster_23299.jpg to edges_bin
Moved edges-cluster_10386.jpg to edges_bin
Moved edges-cluster_21815.jpg to edges_bin
Moved edges-cluster_2866.jpg to edges_bi

In [ ]:
len(os.listdir('edges'))

6749

### Set the Configuration

In [ ]:
# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set WD
os.chdir('/content/drive/My Drive/2024_Data_Science_Project')

# Set configuration
snapshot = "models/wide_resnet38_deeplab_vistas.pth.tar"
data = "segmentation/edges"
output = "segmentation/edges_img_output"
pred_output = "segmentation/edges_pred_output"
prob_output = "segmentation/edges_prob_output"
scales = "[1, 2, 2.7, 3.5]"
do_flip = False
output_mode = "palette"
fusion_mode = "mean"
world_size = 1
rank = 0

### Initiate the Model and DataLoader

In [ ]:
# Create model by loading a snapshot
body, head, cls_state = load_snapshot(snapshot)
model = SegmentationModule(body, head, 256, 65, fusion_mode)
model.cls.load_state_dict(cls_state)
model = model.to(device).eval()
print(model)

# Create data loader
transformation = SegmentationTransform(
    640,
    (0.41738699, 0.45732192, 0.46886091),
    (0.25685097, 0.26509955, 0.29067996),
)
dataset = SegmentationDataset(data, transformation)
data_loader = DataLoader(
    dataset,
    batch_size=1,
    pin_memory=True,
    sampler=DistributedSampler(dataset, world_size, rank),
    num_workers=2,
    collate_fn=segmentation_collate,
    shuffle=False,
)

--- Loading model from snapshot
SegmentationModule(
  (body): WiderResNetA2(
    (mod1): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (pool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (mod2): Sequential(
      (block1): IdentityResidualBlock(
        (bn1): InPlaceABN(64, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
        (convs): Sequential(
          (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): InPlaceABN(128, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (proj_conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (block2): IdentityResidualBlock(
        (bn1): InPlaceABN(128, eps=1e-05, momentum=0.1, affine=True, activation=leaky_

### Run the Image Segmentation

In [ ]:
scales = eval(scales)

with torch.no_grad():
    for batch_i, rec in enumerate(data_loader):
        print("Testing batch [{:3d}/{:3d}]".format(batch_i + 1, len(data_loader)))

        img = rec["img"].to(device)
        probs, preds = model(img, scales, do_flip)

        for i, (prob, pred) in enumerate(
            zip(torch.unbind(probs, dim=0), torch.unbind(preds, dim=0))
        ):
            out_size = rec["meta"][i]["size"]
            img_name = rec["meta"][i]["idx"]

            # Save prediction image
            prob = prob.cpu()
            pred = pred.cpu()
            pred_img = get_pred_image(pred, out_size, output_mode == "palette")
            pred_img.save(path.join(output, img_name + ".png"))

            # Save prediction tensor
            pred = pred.to(torch.uint8)
            tensor_filename = path.join(pred_output, img_name + "_pred.pt")
            save_compressed_tensor(pred, tensor_filename)

            # Save probability tensor
            #prob = prob.to(torch.float16)
            #tensor_filename = path.join(prob_output, img_name + "_prob.pt")
            #save_compressed_tensor(prob, tensor_filename)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Testing batch [1750/6749]
Testing batch [1751/6749]
Testing batch [1752/6749]
Testing batch [1753/6749]
Testing batch [1754/6749]
Testing batch [1755/6749]
Testing batch [1756/6749]
Testing batch [1757/6749]
Testing batch [1758/6749]
Testing batch [1759/6749]
Testing batch [1760/6749]
Testing batch [1761/6749]
Testing batch [1762/6749]
Testing batch [1763/6749]
Testing batch [1764/6749]
Testing batch [1765/6749]
Testing batch [1766/6749]
Testing batch [1767/6749]
Testing batch [1768/6749]
Testing batch [1769/6749]
Testing batch [1770/6749]
Testing batch [1771/6749]
Testing batch [1772/6749]
Testing batch [1773/6749]
Testing batch [1774/6749]
Testing batch [1775/6749]
Testing batch [1776/6749]
Testing batch [1777/6749]
Testing batch [1778/6749]
Testing batch [1779/6749]
Testing batch [1780/6749]
Testing batch [1781/6749]
Testing batch [1782/6749]
Testing batch [1783/6749]
Testing batch [1784/6749]
Testing batch [1785/674

In [ ]:
#pred = load_compressed_tensor("edges_pred_output/edges-cluster_109_pred.pt")